In [1]:
from pathlib import Path
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import os
import shutil
import sklearn.cluster as cl
from sklearn.preprocessing import StandardScaler
os.chdir('/home/weedsci/matt/SemiF-AnnotationPipeline')
from semif_utils.utils import get_cutout_meta

In [18]:
# Get all json files
path = "data/semifield-cutouts/NC_2022-07-14.csv"
df = pd.read_csv(path)
df["is_green"] = np.where(df.is_green==True, 1, 0).astype(float)
df["is_primary"] = np.where(df.is_primary==True, 1, 0).astype(float)
df["extends_border"] = np.where(df.extends_border==True, 1, 0).astype(float)
df["green_sum"] = df.green_sum.astype(float)
df["common_name"].unique()

array(['Sicklepod', 'Cocklebur', 'Palmer amaranth', 'Soybean',
       'Goosegrass', 'Purple nutsedge', 'Common ragweed',
       'Broadleaf signalgrass'], dtype=object)

In [19]:
trn = [
       'area',
       'axis_major_length', 'axis_minor_length', 'centroid0', 'centroid1',
       'eccentricity', 'solidity', 
       'perimeter', 'green_sum']

trndf = df[trn].values.astype(float)
trn_arr = StandardScaler().fit_transform(trndf)
# cls = cl.KMeans(n_clusters=2, random_state=42).fit(trn_arr)
# cls = cl.DBSCAN(eps=0.3)
cls = cl.OPTICS(min_samples=7, xi=0.05, min_cluster_size=0.1)
cls = cls.fit(trn_arr)
if hasattr(cls, "labels_"):
       labels = cls.labels_.astype(int)
else:
       labels = cls.predict(trn_arr)
# labels = cls.labels_
res = pd.DataFrame(data=labels, columns=["cluster"], index=df.index)
res["common_name"] = df["common_name"]
df["cluster"] = res["cluster"]

# ############## Round 2
# df = df[df["cluster"]==-1]
# trndf = df[trn].values.astype(float)
# trn_arr = StandardScaler().fit_transform(trndf)
# # cls = cl.KMeans(n_clusters=2, random_state=42).fit(trn_arr)
# # cls = cl.DBSCAN(eps=0.3)
# cls = cl.OPTICS(min_samples=7, xi=0.05, min_cluster_size=0.1)
# cls = cls.fit(trn_arr)
# if hasattr(cls, "labels_"):
#        labels = cls.labels_.astype(int)
# else:
#        labels = cls.predict(trn_arr)
# # labels = cls.labels_
# res = pd.DataFrame(data=labels, columns=["cluster"], index=df.index)
# res["common_name"] = df["common_name"]
# df["cluster"] = res["cluster"]
# print(df.shape)
df["path"] = df["blob_home"] + "/" + df["data_root"] + "/" + df["cutout_path"]

In [50]:
# dst  = "data/semifield-cutouts/NC_2022-07-14_trash"
# anlyz = df.copy()
# anlyz = anlyz[anlyz["common_name"]=="Soybean"]
# anlyz = anlyz[anlyz["axis_major_length"]<1200]
# anlyz = anlyz[anlyz["axis_minor_length"]<750]
# anlyz = anlyz[anlyz["green_sum"]<5000]
# anlyz = anlyz[anlyz["area"]<5000]
# print(anlyz["cluster"].unique())
# print(df.groupby(["common_name"])["cluster"].count())
# clust0 = anlyz["cluster"]==0
# clust1 = anlyz["cluster"]==1

# if len(clust0) < len(clust1):
#     anlyz = anlyz[anlyz["cluster"]==0]
#     print("cluster: ", "0")
# else:
#     anlyz = anlyz[anlyz["cluster"]==1]
    # print("cluster: ", "1")



[-1  0]
common_name
Broadleaf signalgrass     600
Cocklebur                 917
Common ragweed            434
Goosegrass                785
Palmer amaranth          1512
Purple nutsedge           636
Sicklepod                1289
Soybean                  1387
Name: cluster, dtype: int64


In [21]:
path = "data/semifield-cutouts/NC_2022-07-14.csv"
df = pd.read_csv(path)
df["is_green"] = np.where(df.is_green==True, 1, 0).astype(float)
df["is_primary"] = np.where(df.is_primary==True, 1, 0).astype(float)
df["extends_border"] = np.where(df.extends_border==True, 1, 0).astype(float)
df["green_sum"] = df.green_sum.astype(float)
df["path"] = df["blob_home"] + "/" + df["data_root"] + "/" + df["cutout_path"]
# morph_ana = df[df["is_primary"]==1]
morph_ana = df[df["common_name"]=="Palmer amaranth"]
# print(morph_ana)
morph_ana = morph_ana[morph_ana["extends_border"]==0]
morph_ana = morph_ana[morph_ana["area"]>1000]
morph_ana = morph_ana[morph_ana["green_sum"]>250000]
# morph_ana = morph_ana[morph_ana["green_sum"]<50000]
dst  = Path("data/semifield-cutouts/NC_2022-07-14_trash/Good large palmer")
dst.mkdir(parents=True, exist_ok=True)

for save_path in morph_ana["path"]:
    print(save_path)
    cut_name = Path(save_path).name
    cut_dst = Path(dst, cut_name)
    shutil.copy2(save_path, cut_dst)

KeyError: 'path'

In [18]:
# Saving images by cluster values
for cluster in df["cluster"].unique():
    new_anlyz = df[df["cluster"]==cluster]
    # new_anlyz = new_anlyz[new_anlyz["green_sum"]<5000]
    new_anlyz["path"] = new_anlyz["blob_home"] + "/" + new_anlyz["data_root"] + "/" + new_anlyz["cutout_path"]
    cluster_dir = Path("/home/weedsci/matt/SemiF-AnnotationPipeline/data/semifield-cutouts/NC_2022-07-14_trash", str(cluster))
    cluster_dir.mkdir(exist_ok=False)
    for save_path in new_anlyz["path"]:
        cut_name = Path(save_path).name
        cut_dst = Path(cluster_dir, cut_name)
        # shutil.copy2(save_path, cut_dst)

/tmp/ipykernel_41034/306687636.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_anlyz["path"] = new_anlyz["blob_home"] + "/" + new_anlyz["data_root"] + "/" + new_anlyz["cutout_path"]
/tmp/ipykernel_41034/306687636.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_anlyz["path"] = new_anlyz["blob_home"] + "/" + new_anlyz["data_root"] + "/" + new_anlyz["cutout_path"]
